In [1]:
import os
os.chdir('..')
os.chdir('..')

def collect(preffix):
    return [v for k,v in globals().items() if k.startswith(preffix)]

from Stocky import Dataset

from Stocky import Resample
from Stocky import Indicator
from Stocky import FloatingIndicator
from Stocky import Signalizer
from Stocky import Filter

from Stocky.Structure import GridSearch

import pandas as pd
pd.options.display.max_rows=2000

# ESTIMATOR
call

In [2]:
class MyEstimator(Estimator):
    def __init__(self):
        super().__init__(vars())
    def call(self,resource):
        return None,None

# DETECTOR

In [4]:
class MyDetector(Detector):
    def __init__(self):
        super().__init__(vars())
        self.to_month = Resample.OHLCV('M')
        
        self.MAs = Indicator.MA([3,6,12])
        self.decrease = Signalizer.Decreasing()
        self.non_continuous = Filter.NonContinuous()
        self.refractory = Filter.Refractory(6)
        
        self.KDJ = Indicator.KDJ(5,3,3)
        
        self.MACD = Indicator.MACD(4,9,3)
        
        self.long_MA = Indicator.MA(24)
        
    def call(self,resource):
        ohlcv = resource.stock.ohlcv
        month_close = self.to_month(ohlcv.close)
        
        MAs = self.MAs(month_close)
        con_1 = self.decrease(MAs)
        con_1 = self.non_continuous(con_1)
        con_1 = self.refractory(con_1)
        
        kdj = self.KDJ(month_close)
        con_2 = kdj['K'] > 50
        
        macd = self.MACD(month_close)
        con_3 = macd['OSC'] > 0
        
        long_ma = self.long_MA(month_close)
        con_4 = month_close > long_ma 
        
        cons = con_1 & con_2 & con_3 & con_4
        return cons

# TRADER

In [5]:
class MyTrader(Trader):
    def __init__(self,first_stage_success_rate,second_stage_success_rate,cut_rate):
        super().__init__(vars())
        
        self.first_stage_success_rate = first_stage_success_rate
        self.second_stage_success_rate = second_stage_success_rate
        self.cut_rate = cut_rate
        
        self.MA = Indicator.MA([5,20,60])
        self.increasing = Signalizer.Increasing(False)
        self.descending = Signalizer.Descending(False)
        
        self.to_month = Resample.OHLCV('M')
        self.month_ma = Indicator.MA(3)
        
    def call(self,resource):
        close = resource.stock.ohlcv.close
        mas = self.MA(close)
        self.danger = self.increasing(mas) & self.descending(mas).all(axis=1)
        
        month_close = self.to_month(close)
        self.baseline = self.month_ma(month_close)
        
    def buy(self,resource,record):
        return True,'next day'
    def cancel(self,resource,record):
        return None
    def success(self,resource,record):
        if record.current.day <= 20:
            return record.buy.price * self.first_stage_success_rate, f'buy.price * {self.first_stage_success_rate}'
        else:
            if 'period>20' not in record.note:
                record.note.append('period>20')
            return record.buy.price * self.second_stage_success_rate, f'buy.price * {self.second_stage_success_rate}'
    def fail(self,resource,record):
        return record.buy.price * self.cut_rate
    def flush(self,resource,record):
        if self.danger.loc[record.current.date]:
            return True, 'danger'
        if record.current.day > 60:
            if 'period>60' not in record.note:
                record.note.append('period>60')
            return True, 'period>60'

# ADJUSTER

In [6]:
class MyAdjuster(Adjuster):
    def __init__(self,valid_period=0,groupby_key=None):
        super().__init__(vars())
        
    def rate(self,keys,ref):
        if (ref.sell_price/ref.buy_price).mean() < 1:
            return 0

In [7]:
data = Dataset.ActiveStocks()

In [ ]:
gridsearch = GridSearch('R0',True)

gridsearch.compile_estimator(MyEstimator).grid()
gridsearch.compile_detector(MyDetector).grid()
gridsearch.compile_trader(MyTrader).grid([1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2,2.5,3,3.5,4,5,6,7,8,9],[1.05,1.1,1.2,1.4,1.5,2,2.5,3,3.5,4,4.5,5,6,7,8,9],[0.6,0.7,0.8,0.9])
gridsearch.compile_adjuster(MyAdjuster).grid()

result = gridsearch.fit(data)

Output()

Output()